# LLM Integration and Development

# Text Processing (with a language model)

Install Required Libraries

In [1]:
pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Import Libraries
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM

In [3]:
#loading specific pre-trained models by name
model_name = "facebook/bart-large-cnn"

In [4]:
# Load tokenizer and model for summarization
summarization_tokenizer = AutoTokenizer.from_pretrained(model_name)
summarization_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [5]:
# Load tokenizer and model for text generation
text_generation_tokenizer = AutoTokenizer.from_pretrained(model_name)
text_generation_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [6]:
#First, let's define a news article for summarization:

news_article = """
More than 50 members of the United Nations have joined the U.S. in pursuing a draft resolution to establish artificial intelligence (AI) safety guidelines. 

U.S. Ambassador Linda Thomas-Greenfield on Thursday read a statement that discussed the draft resolution titled "Seizing the Opportunities of Safe, Secure, and Trustworthy Artificial Intelligence Systems for Sustainable Development," which would aim to "articulate a shared approach to AI systems." 

"The resolution calls on Member States to promote safe, secure, and trustworthy AI systems to address the world’s greatest challenges, including those related to poverty elimination, global health, food security, climate, energy, and education," Thomas-Greenfield said in a prepared statement. 
"We are resolved to bridge the artificial intelligence and other digital divides between and within countries through capacity building, increasing digital literacy, and other actions," she added. 
"""


In [7]:
#Now, we can perform text summarization using the BART model:

# Text Summarization
summarization = pipeline("summarization", model=summarization_model, tokenizer=summarization_tokenizer)
summary = summarization(news_article, max_length=150, min_length=30, do_sample=False)

print("Summary:", summary[0]['summary_text'])


Summary: Draft resolution would aim to "articulate a shared approach to AI systems" U.S. Ambassador Linda Thomas-Greenfield: "We are resolved to bridge the artificial intelligence and other digital divides between and within countries"


This will generate a summary of the news article.
Next, let's generate some creative text variations based on a seed sentence:

In [8]:
# Seed sentence
seed_sentence = "In a world where technology reigns supreme,"

In [9]:
# Text Generation
text_generation = pipeline("text-generation", model=text_generation_model, tokenizer=text_generation_tokenizer)
generated_text = text_generation(seed_sentence, max_length=100, num_return_sequences=3)

The model 'BartForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'Pega

In [10]:
print("Generated Text Variations:")
for i, text in enumerate(generated_text):
    print(f"Variation {i+1}: {text['generated_text']}")

Generated Text Variations:
Variation 1: In a world where technology reigns supreme,echnology reign’s supreme. a world in which technology is supreme, and where technology is the only thing that matters. in this world, technology is king, and technology is a way of life. in a world of technology, technology rules supreme, with no limits.
Variation 2: In a world where technology reigns supreme,echnology reign’s supreme. a world in which technology is supreme, and where technology is the only thing that matters. in this world, technology is king, and technology is a way of life. in a world of technology, technology will rule supreme.
Variation 3: In a world where technology reigns supreme,echnology reign’s supreme. a world in which technology is supreme, and where technology is the only thing that matters. in this world, technology is king, and technology is a way of life. in a world of technology, technology rules supreme.


# Custom Prompt Engineering

# Use of Language Model:

Utilizing the BART model from the transformers library to generate responses based on the provided prompts. This showcases how prompts can influence the language model's output to generate contextually relevant responses.

In [11]:
#Import Libraries and Load BART Model
import random
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")


# Clear and Effective Prompts:

 Our prompts dictionary contains clear and concise prompts for various scenarios such as general inquiries, technical support, feature requests, etc. These prompts are designed to elicit specific types of responses from the model

In [12]:
prompts = {
    "General Inquiry": "Hello! How can I assist you today with our software products?",
    "Technical Support": "Are you experiencing any technical issues with our software? Please describe the problem in detail.",
    "Feature Requests": "Do you have any suggestions or feature requests for our software products? We value your feedback!",
    "Product Information": "Would you like more information about a specific product or feature? Please specify, and I'll provide the details.",
    "Account Inquiry": "For account-related inquiries or assistance with licensing, please provide your account details or license key.",
    "Feedback": "We strive to provide excellent service! How would you rate your experience with our support today? Your feedback helps us improve."
}

# Steering Model Output

By providing different prompts for various scenarios, effectively steering the model's output towards generating responses tailored to each specific use case. For example, the prompt for technical support will likely result in responses focused on troubleshooting and problem-solving.

In [13]:
def generate_response(prompt):
    input_text = prompt
    encoded_prompt = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=False)

    # Generate response
    output = model.generate(**encoded_prompt, max_length=100, num_return_sequences=1, temperature=0.7, top_k=50)

    # Decode and return the response
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example Usage:

The generate_response function takes a prompt as input, generates a response from the BART model, and returns the decoded response. This demonstrates how prompts can be used to interact with the language model and obtain responses customized to different scenarios.

In [14]:
# Example usage
for prompt_type, prompt_text in prompts.items():
    print(prompt_type + ":\n" + prompt_text)
    response = generate_response(prompt_text)
    print("Bot Response:")
    print(response)
    print("-------------------------------")


General Inquiry:
Hello! How can I assist you today with our software products?


C:\Users\msarv\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Bot Response:
Hello! How can I assist you today with our software products? Please email us at support@dailymail.co.uk for help with your software products. We are happy to help you with any questions you may have about our products. Please send us an email with your questions and we'll try to assist you.
-------------------------------
Technical Support:
Are you experiencing any technical issues with our software? Please describe the problem in detail.
Bot Response:
Are you experiencing any technical issues with our software? Please describe the problem in detail. We will be happy to help you with any questions you may have about our software. Back to Mail Online home.back to the page you came from. Click here for more information.
-------------------------------
Feature Requests:
Do you have any suggestions or feature requests for our software products? We value your feedback!
Bot Response:
Do you have any suggestions or feature requests for our software products? We value your feedb